# Создание простой RAG системы

Я реализую простую RAG систему как чат с электронным психологом. То есть за данные берем известные книги по психологии

## Шаг 1: Препроцессинг данных и создание эмбедингов
План:
1. Импорт pdf файлов
2. Препроцессинг текста для эмбедингов (разделение на чанки)
3. Эмбендинг чанков текста с помощью эмбендинговой модели
4. Сохранение эмбендингов в файл

## 1. Импорт pdf файлов

In [1]:
import os
import requests

pdf_path = 'psichology books/'
books = ['Allana-Piza-YAzyk-telodvizhenij', 'Eric-Berne-games-that-people-play', 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ', 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi', 'Psihology-Aykido', 'Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha']

for book in books:
    if os.path.exists(pdf_path + book + '.pdf'):
        print(f'[INFO] File exists. Skipping {book}...')
    else:
        print(f'[INFO] {book} is not found')



[INFO] File exists. Skipping Allana-Piza-YAzyk-telodvizhenij...
[INFO] File exists. Skipping Eric-Berne-games-that-people-play...
[INFO] File exists. Skipping Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ...
[INFO] File exists. Skipping Gurina-Koshenova-Rabota-psikhologa-s_detskoi...
[INFO] File exists. Skipping Psihology-Aykido...
[INFO] File exists. Skipping Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha...


In [2]:
 #\xa0

In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text : str) -> str:
    """Performs basic text formatting."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(file_path: str) -> list[str]:
    doc = fitz.open(file_path)
    pages_and_texts = []
    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text("text")
        text = text_formatter(text=text)
        if "Goleman" in file_path:
            text = text.replace("\xa0", " ")
        pages_and_texts.append({"book": file_path[17:-4], "page_num": page_num, "page_char_count": len(text), "page_word_count": len(text.split()), "page_token_count": len(text)/4, "text": text})
    return  pages_and_texts

texts = []
for book in books:
    texts.extend(open_and_read_pdf(pdf_path + book + '.pdf'))
len(texts)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

1748

In [4]:
# Дополнительный препроцессинг текста

import re
from nltk import sent_tokenize
from nltk.corpus import stopwords
import pymorphy3  # pip install pymorphy3

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def lemmatize(text: str) -> str:
    return ' '.join(morph.parse(word)[0].normal_form for word in text.split())

def advanced_text_formatter(text: str) -> str:
    """Enhanced text formatting beyond basic."""
    # Step 1: Remove hyphens from word breaks
    text = re.sub(r'(\\w+)-\\s+(\\w+)', r'\\1\\2', text)
    
    # Step 2: Normalize multiple spaces
    text = re.sub(r'\\s+', ' ', text)
    
    # Step 3: Remove specific annotations like footnotes
    text = re.sub(r'—\\s*Примеч\\.\\s*пер\\.\\s*', '', text)
    text = re.sub(r'\\(\\s*\\d+\\s*\\)\\.', '', text)
    
    # Lemmatize and lowercase
    text = lemmatize(text.lower())  # Добавьте lowercase для унификации
    
    # Remove stops AFTER lemmatize (закомментировано: лучше для эмбеддингов)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    if len(text.strip()) == 0:  # Проверка на пустой текст
        return ""
    
    return text.strip()

# Fallback sentence splitter if NLTK not available
def custom_sent_tokenize(text: str) -> list[str]:
    """Custom regex-based sentence tokenizer for Russian text."""
    sentence_end = re.compile(r'(?<!\\w\\.\\w.)(?<![A-ZА-Я][a-zа-я]\\.)(?<=\\.|\\?|\\!|\\…)\\s')
    sentences = sentence_end.split(text)
    return [s.strip() for s in sentences if s.strip()]

# Apply advanced formatting to existing texts
for item in texts:
    item['text'] = advanced_text_formatter(item['text'])

# Now split into sentences
sentences = []
for item in texts:
    if not item['text']:  # Пропуск пустых
        continue
    # Use NLTK if available
    try:
        page_sentences = sent_tokenize(item['text'], language='russian')
    except:
        page_sentences = custom_sent_tokenize(item['text'])
    
    for sent in page_sentences:
        # Step 6: Trim leading/trailing punctuation if unnecessary
        sent = re.sub(r'^[^\\w]+', '', sent)  # Remove leading non-word chars
        sent = re.sub(r'[^\\w]+$', '', sent)  # Remove trailing non-word chars
        
        sentences.append({
            "book": item['book'],
            "page_num": item['page_num'],
            "sentence": sent,
            "sent_char_count": len(sent),
            "sent_word_count": len(sent.split()),
            "sent_token_count": len(sent) / 4
        })

print(f"Total sentences extracted: {len(sentences)}")

# Optionally, save to file or further process
import json
with open('processed_sentences.json', 'w', encoding='utf-8') as f:
    json.dump(sentences, f, ensure_ascii=False, indent=4)

Total sentences extracted: 26184


In [5]:
texts[700]['text']

'психика медицина 281 глава 11 психика медицина «кто научить всему, доктор?» ответ последовать незамедлительно: «страдание». альбер камю . чума непонятный тупой боль пах погнать больницу. осмотр выявить необычного, пока врач увидеть результат анали- мочи: обнаружить след крови. — хочу, лечь больница пройти кое- иссле- дования. проверить работа почек, сделать цитологию… — сказать деловой тоном. помню, говорить дальше. сознание, казалось, засты- ло слово «цитология». рак. остаться смутный воспоминание, объяснял, должный пройти диагностику. простейшее указания, прийтись просить несколько повторить. «цитология…» — ум желать расставаться это словом. оно вызвать чувство, сзади схватить горло грабить порог соб- ственный дома.'

In [6]:
import pandas as pd

df = pd.DataFrame(texts)
df.head()


,book,page_num,page_char_count,page_word_count,page_token_count,text
0,Allana-Piza-YAzyk-telodvizhenij,0,0,0,0.00,
1,Allana-Piza-YAzyk-telodvizhenij,1,1378,182,344.50,annotation книга аллан пиза «язык телодвижений...
2,Allana-Piza-YAzyk-telodvizhenij,2,1021,138,255.25,различие пространственный зона горожанин жител...
3,Allana-Piza-YAzyk-telodvizhenij,3,1052,142,263.00,"жест скрещивание рук, усиленный сжатие палец к..."
4,Allana-Piza-YAzyk-telodvizhenij,4,902,112,225.50,"жесты, использовать мужчина ухаживание жест си..."


In [7]:
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count
count,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90
std,135.44,618.48,93.74,154.62
min,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56
50%,161.00,1939.00,264.00,484.75
75%,275.00,2102.00,288.00,525.50
max,544.00,3273.00,1386.00,818.25


In [8]:
from spacy.lang.ru import Russian
nlp = Russian()

nlp.add_pipe("sentencizer")

doc = nlp('Это первое предложение. Это второе предложение! А это третье предложение?')
assert len(list(doc.sents)) == 3

list(doc.sents)

[Это первое предложение., Это второе предложение!, А это третье предложение?]

In [9]:
for item in tqdm(texts):
    item['sentences'] = list(nlp(item['text']).sents)

    item['sentences'] = [str(sent) for sent in item['sentences']]

    item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/1748 [00:00<?, ?it/s]

In [10]:
import random
random.sample(texts, k=1)

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 213,
  'page_char_count': 0,
  'page_word_count': 0,
  'page_token_count': 0.0,
  'text': '',
  'sentences': [],
  'page_sentence_count_spacy': 0}]

In [11]:
data = pd.DataFrame(texts)
data.describe().round(2)


,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,15.76
std,135.44,618.48,93.74,154.62,11.12
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,111.00


In [12]:
# chunking with spacy sentences (1 chunk = ?10? sentences). 

num_sentences_per_chunk = 5 # пробовала 10, 7

def split_list(input_list: list[str], slice_size: int=num_sentences_per_chunk) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [13]:
import re


all_sentences = []
for item in tqdm(texts):
    for sent in item['sentences']:
        all_sentences.append({
            "sentence": str(sent),
            "book": item['book'],
            "page_num": item['page_num']
        })


sentence_chunks = split_list(all_sentences, slice_size=num_sentences_per_chunk)


pages_and_chunks = []
for chunk in tqdm(sentence_chunks):
    chunk_dict = {}
    chunk_dict['book'] = chunk[0]['book']
    chunk_dict['page_num'] = chunk[0]['page_num']
    
    # Join the sentences in the chunk
    joined_sentence_chunk = " ".join([item['sentence'] for item in chunk]).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
    chunk_dict['sentence_text'] = joined_sentence_chunk
    
    # Recalculate stats for the new chunk
    chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
    chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
    chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4
    
    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/5512 [00:00<?, ?it/s]

5512

In [14]:
random.sample(texts, k=1)

[{'book': 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi',
  'page_num': 40,
  'page_char_count': 1906,
  'page_word_count': 255,
  'page_token_count': 476.5,
  'text': '41 ектом (событием) общий схемой. восприятие рассматривает- ся процесс генерализация объект конкретизация схема [42]. мнение д. нормана, схема – это некий пакет информации, определённый комплекс знаний, касаться какой-либо области. схема включать значительный сведение соответствующий понятиях, число типичный особенность обозначать объектов. так, схема «животное» констатирует, это организм голова, схема «млекопитающее» – организм че- тырь конечность т. д. [47]. д. румельхарт д. норман схема понимать единица организовать информации, взаимосвязанный структура знания, который принимать участие понимание входной информация управлять процесс обработки. этап, называть «структурирование знания», включать формирование новый схема основа существующий путём видоизменение последний вли- яние новый опыт [42]. дж. брунер формулироват

In [15]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,15.76
std,135.44,618.48,93.74,154.62,11.12
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,111.00


In [16]:
import re
from tqdm.auto import tqdm

num_sentences_per_chunk = 10
def split_list(input_list: list, slice_size: int = num_sentences_per_chunk) -> list[list]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(texts):
    if 'sentences' in item:
        item['sentence_chunks'] = split_list(item['sentences'])
        item['num_chunks'] = len(item['sentence_chunks'])

pages_and_chunks = []
for item in tqdm(texts):
    if 'sentence_chunks' in item:
        for sentence_chunk in item['sentence_chunks']:
            chunk_dict = {}
            chunk_dict['book'] = item['book']
            chunk_dict['page_num'] = item['page_num']
            joined_sentence_chunk = ' '.join(map(str, sentence_chunk)).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
            chunk_dict['sentence_text'] = joined_sentence_chunk
            
            chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
            chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
            chunk_dict['chunk_token_count'] = len(joined_sentence_chunk)/4

            pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/1748 [00:00<?, ?it/s]

3596

In [17]:
random.sample(pages_and_chunks, k=1)

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 178,
  'sentence_text': 'курение процесс курение внешний проявление внутренний дисба-ланс конфликт внутри человека, иметь слишком мало общий привязанность поглощение никотина. это способов, который пользоваться современный человек условие повышенный стресса, ослабление напряжение свой нервов, который накапли-ваться процесс социальный деловой контактов. например, большинство человек испытывать внутренний напряжение, сидя приёмный дантист ожидание удаление зуба. курильщик сгладить свой напряжение помощь курения, некурящий проделывать ритуальный жесты. отряхивать одежду, грызть ногти, стучать палец ногами, поправлять запонки, почёсывать затылок, снимать одевать кольцо пальца, вертеть рука галстук демонстрировать множество жестов, который говорить том, человек нервничает, требоваться поддержка.',
  'chunk_char_count': 776,
  'chunk_word_count': 86,
  'chunk_token_count': 194.0}]

In [18]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1748.00,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,15.76,2.06
std,135.44,618.48,93.74,154.62,11.12,1.13
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00,1.00
50%,161.00,1939.00,264.00,484.75,14.00,2.00
75%,275.00,2102.00,288.00,525.50,18.00,2.00
max,544.00,3273.00,1386.00,818.25,111.00,12.00


In [19]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_num,chunk_char_count,chunk_word_count,chunk_token_count
count,3596.00,3596.00,3596.00,3596.00
mean,194.82,717.56,86.49,179.39
std,143.75,433.65,50.45,108.41
min,0.00,2.00,1.00,0.50
25%,72.00,359.00,46.00,89.75
50%,163.00,660.50,81.00,165.12
75%,292.00,1042.25,123.00,260.56
max,543.00,2042.00,276.00,510.50


In [20]:
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():

    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_text"]}')

Chunk token count: 26.25 | Text: под- тверждение стать результат исследования, провести начальный школа использование нейропсихологический
Chunk token count: 18.5 | Text: зависимость обстоятельство мочь присоединяться эмоциональный поведенческий
Chunk token count: 15.0 | Text: искусство общение 223 часть iii эмоциональный разум действие
Chunk token count: 15.25 | Text: глава угол поставить основный способность, который невозможно
Chunk token count: 14.5 | Text: темперамент приговор 383 часть v эмоциональный грамотность


In [21]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pages_and_chunks_over_min_token_len[20:25]

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 17,
  'sentence_text': 'конгруэнтность — совпадение слово жест собеседник человека, показать рис. 4, попросить выразить свой мнение относительно того, сказали, ответил, согласен, невербальный сигнал конгруэнтными, т.е. соответ-ствовали словесный высказываниям. скажет, очень нравиться все, говорите, лгать, слово жест конгруэнтными. исследование доказывают, невербальный сигнал нести 5 большой информации, вербальные, случае, сигнал — конгруэнтны, человек полагаться невербальный информацию, предпочитать словесной. часто наблюдать, какой-нибудь политик стоить трибуне, крепко скрестить рука грудь ( защитный поза ) опустить подбородок ( критический враждебный поза), говорить аудитория том, восприимчивый дружелюбно относиться идея молодежи. мочь попытаться убедить аудитория свой теплом, гуманный отношении, делать быстрые, резкий удар трибуне. зигмунд фрейд однажды заметил, пациентка словесно убеждать том, счастливый браке, бессознатель

In [22]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'book': 'Eric-Berne-games-that-people-play',
  'page_num': 139,
  'sentence_text': 'обра- зом выигрывать любой случае. блэк делает, это чувство беспомощности, что-нибудь делает, раздражения. поэтому человек склонный разыгрывать игра “а почему не. . . да, но. . . ”, доставлять удовле- творение мягкий форме. лёгкий решение видно, искать, пока хороший понять психодинамика игры. 7. деревянный нога наиболее драматический форма “деревянный ноги” — “ссылка ненормальность”. язык анализ взаимодействие это пере- водиться так: “чего хотеть человек расстроенны- ми эмоциями, меня, — мочь удержаться убий- ства!” это ожидаться ответ суда: “конечно, нет: мочь требовать этого!” “',
  'chunk_char_count': 587,
  'chunk_word_count': 82,
  'chunk_token_count': 146.75}]

## Эмбендинг

In [ ]:
import requests

# "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

model_id =  'Geotrend/LaBSE-en-ru-v2'#hugging face model
hf_token = "get your token in http://hf.co/settings/tokens"

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path=model_id, device='cpu')

sentences = ["Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.", 
             "Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.",
             "Мне нравятся лошади"]




embeddings = embedding_model.encode(sentences)
embeddings_dictionary = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dictionary.items():
    print(f'Sentence: {sentence}\nEmbedding: {embedding[:5]}\n')

c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sentence: Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.
Embedding: [ 0.09367365  0.06438734  0.15397337  0.27672583 -0.28899655]

Sentence: Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.
Embedding: [-0.18342024 -0.00970362  0.11679667  0.56002    -0.312298  ]

Sentence: Мне нравятся лошади
Embedding: [ 0.09567402  0.13053127  0.00370111  0.27585265 -0.23311627]



In [24]:
embeddings[0].shape

(384,)

%%time

embedding_model.to('cpu')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

In [25]:
%%time

embedding_model.to('cuda')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

  0%|          | 0/3371 [00:00<?, ?it/s]

CPU times: total: 44.6 s
Wall time: 42.8 s


In [26]:
%%time
embedding_model.to('cuda')

text_chunks = [item["sentence_text"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: total: 0 ns
Wall time: 3 ms


'внешний общественное. приводиться ключевой вы- ражение, применять аналогичный игра развлечении, разыг- рывать менее интимный обстановке. ( 5). биологическое. делаться попытка охарактеризовать вид поглаживания, ко- торой игра доставлять участникам. ( 6). экзистенциальное. указываться позиция, который игра разыгрываться типичный случаях. родственный игры: приводиться название дополнитель- ных, сходный противоположный (антитетических) игр. точный понимание игра мочь достигнуть лишь услови- яха психотерапии.'

In [27]:
len(text_chunks)

3371

In [28]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

text_chunk_embeddings

CPU times: total: 8.25 s
Wall time: 3.6 s


tensor([[ 0.0397, -0.2058, -0.1221,  ...,  0.1773, -0.1527, -0.0165],
        [ 0.0901,  0.0057,  0.1217,  ...,  0.2975, -0.1015,  0.2282],
        [-0.0460,  0.0458,  0.1394,  ...,  0.1055, -0.0223,  0.2804],
        ...,
        [-0.0650,  0.0390, -0.2216,  ...,  0.2135, -0.0566,  0.1209],
        [-0.0245,  0.0519, -0.1911,  ...,  0.1515, -0.0351,  0.1642],
        [-0.0024, -0.1104, -0.3956,  ..., -0.1087, -0.2172,  0.2494]],
       device='cuda:0')

## Сохранение эмбеддингов в файл

In [29]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [30]:
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df.head()

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,annotation книга аллан пиза «язык телодвижений...,1181,134,295.25,[ 3.96876633e-02 -2.05777466e-01 -1.22056380e-...
1,Allana-Piza-YAzyk-telodvizhenij,2,различие пространственный зона горожанин жител...,943,115,235.75,[ 9.01253074e-02 5.71551407e-03 1.21663198e-...
2,Allana-Piza-YAzyk-telodvizhenij,3,"жест скрещивание рук, усиленный сжатие палец к...",961,121,240.25,[-0.04603301 0.04575719 0.13937409 -0.031644...
3,Allana-Piza-YAzyk-telodvizhenij,4,"жесты, использовать мужчина ухаживание жест си...",818,99,204.50,[-0.02531953 0.17106412 -0.06040524 0.074408...
4,Allana-Piza-YAzyk-telodvizhenij,5,ступень нога выражать заинтересованность распо...,704,89,176.00,[ 2.25336701e-01 1.51010826e-01 -4.16030847e-...


Если бы у меня была действительно большая база данных эмбэндингов (100K - 1M) то тогда бы использовала векторную базу данных qdrant

# Поиск и ответы

In [31]:
import random
import torch
import numpy as np
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")


text_chunks_and_embeddings_df['embedding'] = text_chunks_and_embeddings_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' '))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df['embedding'].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient='records')

text_chunks_and_embeddings_df

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,annotation книга аллан пиза «язык телодвижений...,1181,134,295.25,"[0.0396876633, -0.205777466, -0.12205638, -0.0..."
1,Allana-Piza-YAzyk-telodvizhenij,2,различие пространственный зона горожанин жител...,943,115,235.75,"[0.0901253074, 0.00571551407, 0.121663198, -0...."
2,Allana-Piza-YAzyk-telodvizhenij,3,"жест скрещивание рук, усиленный сжатие палец к...",961,121,240.25,"[-0.04603301, 0.04575719, 0.13937409, -0.03164..."
3,Allana-Piza-YAzyk-telodvizhenij,4,"жесты, использовать мужчина ухаживание жест си...",818,99,204.50,"[-0.02531953, 0.17106412, -0.06040524, 0.07440..."
4,Allana-Piza-YAzyk-telodvizhenij,5,ступень нога выражать заинтересованность распо...,704,89,176.00,"[0.225336701, 0.151010826, -0.0416030847, -0.0..."
...,...,...,...,...,...,...,...
3366,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,404,новый издание — почему оно актуально вас? разм...,319,39,79.75,"[-0.117484666, 0.0297225341, -0.0382157266, 0...."
3367,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,убийство вирджиния северный иллинойсе). 2. доп...,852,94,213.00,"[-0.02763486, -0.11024814, -0.05699018, -0.105..."
3368,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,"удвоиться количество отчёт читателей, рассказ ...",555,63,138.75,"[-0.0649724379, 0.039032992, -0.221619144, 0.1..."
3369,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,406,"деятельность особенность заказчиков, дать возм...",1003,114,250.75,"[-0.0244564228, 0.0519208908, -0.191072792, 0...."


In [32]:
random.sample(list(text_chunks_and_embeddings_df['sentence_text']), k=1)

['примерно од- ный ребёнок десять это возникать определённый трудности10. проблема мочь составлять плохой ощущение личный простран- ства, ребёнок время разговор вставать слишком близко собе- седник разбрасывать свой вещь личный территория людей; неумение расшифровывать язык телодвижение пользо- ваться им; неправильный истолкование использование выраже- ний лица, примеру, неумение устанавливать зрительный контакт неспособность правильно расставить акцент отрегулировать']

In [33]:
embeddings.shape

torch.Size([3371, 384])

In [34]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer(model_name_or_path=model_id, device='cpu')

c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
from time import perf_counter as timer

query = 'На чем строится доверие между людьми'
print(f'Query: {query}')

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

print(query_embedding.shape)


cosine_scores = util.cos_sim(query_embedding, embeddings)[0]

start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores: {end_time - start_time:.4f} seconds \n len = {len(embeddings)}')

top_results = torch.topk(cosine_scores, k=5)
top_results

Query: На чем строится доверие между людьми
torch.Size([384])
[INFO] Time taken to compute dot product scores: 0.0001 seconds 
 len = 3371


torch.return_types.topk(
values=tensor([0.6723, 0.6044, 0.6013, 0.5934, 0.5917], device='cuda:0'),
indices=tensor([1232, 2901, 3081, 3069, 2264], device='cuda:0'))

In [43]:
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import MinMaxScaler
from nltk.tokenize import word_tokenize  # Для токенизации

# Токенизируйте чанки заранее (используйте DF из [30] для удобства; если нет, замените на pages_and_chunks_over_min_token_len)
# tokenized_chunks = [word_tokenize(chunk.lower()) for chunk in text_chunks_and_embeddings_df['sentence_text']]  # Если DF загружен

tokenized_chunks = [word_tokenize(chunk['sentence_text'].lower()) for chunk in pages_and_chunks_over_min_token_len]  # Для списка

bm25 = BM25Okapi(tokenized_chunks)

# Для запроса
tokenized_query = word_tokenize(query.lower())
bm25_scores = bm25.get_scores(tokenized_query)

# Гибрид: Нормализуйте и комбинируйте (alpha=0.5 для баланса)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Переносим cosine_scores на CPU и конвертируем в numpy
cosine_scores_cpu = cosine_scores.cpu().numpy()  # Добавьте эту строку

# Теперь используем CPU-версию
norm_cos = scaler.fit_transform(cosine_scores_cpu.reshape(-1, 1)).flatten()

# Для bm25_scores (если это list, преобразуем в numpy)
bm25_scores_np = np.array(bm25_scores)  # На всякий случай
norm_bm25 = scaler.fit_transform(bm25_scores_np.reshape(-1, 1)).flatten()


# hybrid_scores = 0.5 * norm_cos + 0.5 * norm_bm25
hybrid_scores = 0.3 * norm_cos + 0.7 * norm_bm25


# Продолжаем как раньше
top_results = torch.topk(torch.tensor(hybrid_scores), k=5)

In [44]:
from sentence_transformers import CrossEncoder
reranker = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')  # Multilingual

# После top_results = torch.topk(cosine_scores, k=20)  # Берите топ-20
top_chunks = [pages_and_chunks[idx]['sentence_text'] for idx in top_results.indices.tolist()]
pairs = [[query, chunk] for chunk in top_chunks]
rerank_scores = reranker.predict(pairs)

# Сортировка и топ-5
reranked = sorted(zip(rerank_scores, top_chunks), key=lambda x: x[0], reverse=True)[:5]
for score, text in reranked:
    print(f"Rerank Score: {score:.4f} | Text: {text}")

c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Rerank Score: -4.6401 | Text: уверенность. чувство контроль владение свой телом, управление свой поведением, доверие миру; ребёнок ощущает, он, скорее всего, суметь сделать то, берет- ся, взрослый помогут. 2. любознательность. ребёнок нравиться узнавать разный но- вых вещах.
Rerank Score: -5.3266 | Text: спо- собность держать свой порыв узда — основа сила воля характе- ра человека. аналогично основа альтруизм лежать эмпатия, умение распознавать эмоция людей. понимание нужда отчаяние человека, беспокоиться чем. мораль- ные установки, который большой весь нуждаться наш общество се- годень , — это самоконтроль сострадание.
Rerank Score: -5.9334 | Text: это человек реальный проблемами; отчаянно хотеть разрешить свой проблемы. поверили, тм помочь найти выход. друг показал, теория, который надеялись, ложная. паника! чтоть сделать немедленно, прежде логика взять свой снова оставить надежды. необходимо быстро возвести стены, укрыться довод рассудка. иметь значения, крепость, который строится, 

In [45]:
for i in top_results.indices.tolist():
    print(text_chunks_and_embeddings_df['sentence_text'][i])

а. фрейд подчеркивает, работа это этап иметь ещё общий действительный аналитический рабо- той, т. е. ещё речь перевод сознание бессознательный процесс аналитический воздействие пациента. подгото- вительный этап иметь цель создать предварительный условия, необ- ходить начало анализа: сознание болезни, доверие решение пройти анализ [66]. 8 1963 г. а. фрейд описать линия развитие ребенка, придать теория завершить вид, однако имя автор идея (г. хут-хельмут) остаться несправедливый забытым. 9 гермин хуг-хельмут убитый свой племянник рольфом, анализ который проводить который причислять «трудным» ребёнок (на момент убийство рольф 18 лет).
спо- собность держать свой порыв узда — основа сила воля характе- ра человека. аналогично основа альтруизм лежать эмпатия, умение распознавать эмоция людей. понимание нужда отчаяние человека, беспокоиться чем. мораль- ные установки, который большой весь нуждаться наш общество се- годень , — это самоконтроль сострадание.
уверенность. чувство контроль владение

In [46]:
larger_embeddings = torch.randn(1000 * embeddings.shape[0], embeddings.shape[1]).to(device)
print(f"Larger embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores with larger embeddings: {end_time - start_time:.4f} seconds \n len = {len(larger_embeddings)}')

Larger embeddings shape: torch.Size([3371000, 384])
[INFO] Time taken to compute dot product scores with larger embeddings: 0.0004 seconds 
 len = 3371000


In [47]:
import textwrap

def print_wrapped(text: str, width: int=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

In [48]:
print(f"Query: {query}\n")
print("Results:")

for score, idx in zip(top_results[0], top_results[1]):
    print_wrapped(f"Score: {score:.4f} | Text: {pages_and_chunks[idx]['sentence_text']}\n")


Query: На чем строится доверие между людьми

Results:
Score: 0.8526 | Text: а. фрейд подчеркивает, работа это этап иметь ещё общий
действительный аналитический рабо- той, т. е. ещё речь перевод сознание
бессознательный процесс аналитический воздействие пациента. подгото- вительный
этап иметь цель создать предварительный условия, необ- ходить начало анализа:
сознание болезни, доверие решение пройти анализ [66]. 8 1963 г. а. фрейд описать
линия развитие ребенка, придать теория завершить вид, однако имя автор идея (г.
хут-хельмут) остаться несправедливый забытым. 9 гермин хуг-хельмут убитый свой
племянник рольфом, анализ который проводить который причислять «трудным» ребёнок
(на момент убийство рольф 18 лет).
Score: 0.8096 | Text: спо- собность держать свой порыв узда — основа сила воля
характе- ра человека. аналогично основа альтруизм лежать эмпатия, умение
распознавать эмоция людей. понимание нужда отчаяние человека, беспокоиться чем.
мораль- ные установки, который большой весь нуждатьс